# 机器学习部分

In [1]:
import pandas as pd
import numpy as np
import jieba
import re
train_file = './data1/data_train1.csv'
test_file = './data1/data_test1.csv'

## 分词

In [4]:
# -*- coding: utf-8 -*-
def seg(path,train):
    oral_csv = pd.read_csv(path,delimiter = '\t',header = None, index_col=False)
    com = oral_csv.loc[:,2]
    docs = np.array(com)

    seged_corpus = []
    if train:
        seged_file = open('./data1/train_seg.txt','w')
    else:
        seged_file = open('./data1/test_seg.txt','w')
    for line in docs:
        line = str(line).strip()
        words = jieba.cut(line)
        words = " ".join(words)
        seged_file.write(words.encode('utf-8'))
        seged_file.write('\n')
        seged_corpus.append(words)

In [5]:
seg(test_file,False)
seg(train_file,True)

## 加入类别信息生成的结果

In [6]:
# -*- coding: utf-8 -*-
from sen_model_cate import *
from sklearn import preprocessing
from util import *
import numpy as np
import pandas as pd
import os
import csv
class CONFIG(object):
    METHOD = 'lr'
    SEED_POS = './data1/seed/sum-positive.txt'  # 正向情感词典
    SEED_NEG = './data1/seed/sum-negative.txt'  # 负向情感词典
    SPT = '\t'
    GEN_FEAT_METHOD = 'seed,unigram,bigram'
    N_FEAT = 5000
    FREQ = False
seed_pos = gen_seed_dic(CONFIG.SEED_POS)
seed_neg = gen_seed_dic(CONFIG.SEED_NEG)
param = {
          'boosting_type': 'gbdt',
          'objective': 'regression_l2',
          'metric': 'mse',
          'min_child_weight': 1.5,
          'num_leaves': 2**5,
          'lambda_l2': 10,
          'subsample': 0.7,
          'colsample_bytree': 0.5,
          'colsample_bylevel': 0.5,
          'learning_rate': 0.1,
          'scale_pos_weight': 20,
          'seed': 2018,
          'nthread': 4,
          'silent': True,
}
test_file = './data1/data_test1.csv'
test_csv = pd.read_csv(test_file,delimiter = '\t',header = None, index_col=False)
test_com = test_csv.loc[:,1].values
test_doc = np.array(test_com)

train_file = './data1/data_train1.csv'
train_csv = pd.read_csv(train_file,delimiter = '\t',header = None, index_col=False)
train_target = train_csv.loc[:,3].values
train_com = train_csv.loc[:,1].values
train_doc = np.array(train_com)

label_encoder = preprocessing.LabelEncoder()  # 标准化标签
label_encoder.fit(train_doc)

train_cate = label_encoder.transform(train_doc)
train_cate = train_cate.reshape((len(train_cate),1))

train_docs = []
with open('./data1/train_seg.txt', 'r') as ipt:
    for line in ipt.readlines():
        items = line.strip().split(' ')
        train_docs.append(items)

test_docs = []
with open('./data1/test_seg.txt', 'r') as ipt:
    for line in ipt.readlines():
        items = line.strip().split(' ')
        test_docs.append(items)

s_model = sen_model(method='lr',
                    docs=train_docs,
                    cate=train_cate,
                    target=train_target,
                    test=test_docs,
                    seed_pos=seed_pos,
                    seed_neg=seed_neg,
                    gen_feat_method=CONFIG.GEN_FEAT_METHOD,
                    n_feat=CONFIG.N_FEAT,
                    freq=CONFIG.FREQ,
                    param=param)
test = s_model.test
test_cate = label_encoder.transform(test_doc)
test_cate = test_cate.reshape((len(test_cate),1))
test = np.hstack((test,test_cate))
lr_result = s_model.predict(test)

/anaconda3/envs/python27/lib/python2.7/site-packages/sklearn/ensemble/weight_boosting.py:29: DeprecationWarning: numpy.core.umath_tests is an internal NumPy module and should not be imported. It will be removed in a future NumPy release.
  from numpy.core.umath_tests import inner1d


ImportError: No module named lightgbm

## 写入预测概率

In [ ]:
with open("./result/result_lr_5000_cate.txt","w") as ipt:
    for line in lr_result:
        for i in line:
            ipt.write(str(i)+" ")
        ipt.write('\n')

## 未加入类别信息生成的结果

In [1]:
# -*- coding: utf-8 -*-
from sen_model import *
from sklearn import preprocessing
from util import *
import numpy as np
import pandas as pd
import os
import csv
class CONFIG(object):
    METHOD = 'lr'
    SEED_POS = './data/seed/sum-positive.txt'
    SEED_NEG = './data/seed/sum-negative.txt'
    SPT = '\t'
    GEN_FEAT_METHOD = 'seed,unigram,bigram'
    N_FEAT = 5000  # 前五千个作为特征
    FREQ = False   # 有ngram特征为1，没有为0    True时是指 频率，出现的次数
seed_pos = gen_seed_dic(CONFIG.SEED_POS)
seed_neg = gen_seed_dic(CONFIG.SEED_NEG)
param = {     #lightgbm 参数
          'boosting_type': 'gbdt',
          'objective': 'regression_l2',
          'metric': 'mse',
          'min_child_weight': 1.5,
          'num_leaves': 2**5,
          'lambda_l2': 10,
          'subsample': 0.7,
          'colsample_bytree': 0.5,
          'colsample_bylevel': 0.5,
          'learning_rate': 0.1,
          'scale_pos_weight': 20,
          'seed': 2018,
          'nthread': 4,
          'silent': True,
}
test_file = './data/data_test1.csv'
test_csv = pd.read_csv(test_file,delimiter = '\t',header = None, index_col=False,encoding='gbk')

train_file = './data/data_train1.csv'
train_csv = pd.read_csv(train_file,delimiter = '\t',header = None, index_col=False,encoding='gbk')
train_target = train_csv.loc[:,3].values

train_docs = []
with open('./data/train_seg.txt', 'r') as ipt:
    for line in ipt.readlines():
        items = line.strip().split(' ')
        train_docs.append(items)

test_docs = []
with open('./data/test_seg.txt', 'r') as ipt:
    for line in ipt.readlines():
        items = line.strip().split(' ')
        test_docs.append(items)

s_model = sen_model(method='lr',
                    docs=train_docs,
                    target=train_target,
                    test=test_docs,
                    seed_pos=seed_pos,
                    seed_neg=seed_neg,
                    gen_feat_method=CONFIG.GEN_FEAT_METHOD,
                    n_feat=CONFIG.N_FEAT,
                    freq=CONFIG.FREQ,
                    param=param)
test = s_model.test
lr_result = s_model.predict(test)

start init...
start gen unigram feat..
gen unigram feat successed..
start init...
start gen bigram feat..
gen bigram feat successed..
start init...
start gen seed feat..
start training model..


/anaconda3/lib/python3.6/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/anaconda3/lib/python3.6/site-packages/sklearn/linear_model/logistic.py:459: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.
  "this warning.", FutureWarning)


start predict...


## 写入预测概率

In [2]:
with open("./result/result_lr_5000.txt","w") as ipt:
    for line in lr_result:
        for i in line:
            ipt.write(str(i)+" ")
        ipt.write('\n')

## 融合生成ronghe_cnnBilstm701_300_100_wiki_lr_cate文件

In [1]:
with open('./result/result_lr_5000_cate.txt') as ipt:
    lr = []
    for line in ipt:
        line = line.strip().split()
        lr.append(line)
with open('./result/result_cnn_Bilstm_unions701_300_100_wiki.txt','r') as f:
    cnn_lstm = []
    for ls in f:
        ls = ls.strip().split()
        cnn_lstm.append(ls)
jg = []
for j in range(len(lr)):
    jg1 = []
    for k in range(len(lr[0])):
        c = float(lr[j][k])+float(cnn_lstm[j][k])
        jg1.append(c)
    jg.append(jg1)
import pandas as pd
import numpy as np
result = []
for i in jg:
    r = i.index(max(i))
    result.append(r)
a = range(1,len(result)+1)
a = np.array(a)
df = pd.DataFrame({'ind':a,'pre':result})
df.to_csv('./result/ronghe_cnnBilstm701_300_100_wiki_lr_cate.csv',index = False,header=False) 

## 融合生成ronghe_cnnBilstm616_lr_cate文件

In [2]:
with open('./result/result_lr_5000_cate.txt') as ipt:
    lr = []
    for line in ipt:
        line = line.strip().split()
        lr.append(line)
with open('./result/result_cnn_Bilstm_616.txt','r') as f:
    cnn_lstm = []
    for ls in f:
        ls = ls.strip().split()
        cnn_lstm.append(ls)
jg = []
for j in range(len(lr)):
    jg1 = []
    for k in range(len(lr[0])):
        c = float(lr[j][k])+float(cnn_lstm[j][k])
        jg1.append(c)
    jg.append(jg1)
import pandas as pd
import numpy as np
result = []
for i in jg:
    r = i.index(max(i))
    result.append(r)
a = range(1,len(result)+1)
a = np.array(a)
df = pd.DataFrame({'ind':a,'pre':result})
df.to_csv('./result/ronghe_cnnBilstm616_lr_cate.csv',index = False,header=False) 

## 融合生成ronghe_cnnBilstm616_lr文件 

In [3]:
with open('./result/result_lr_5000.txt') as ipt:
    lr = []
    for line in ipt:
        line = line.strip().split()
        lr.append(line)
with open('./result/result_cnn_Bilstm_616.txt','r') as f:
    cnn_lstm = []
    for ls in f:
        ls = ls.strip().split()
        cnn_lstm.append(ls)
jg = []
for j in range(len(lr)):
    jg1 = []
    for k in range(len(lr[0])):
        c = float(lr[j][k])+float(cnn_lstm[j][k])
        jg1.append(c)
    jg.append(jg1)
import pandas as pd
import numpy as np
result = []
for i in jg:
    r = i.index(max(i))
    result.append(r)
a = range(1,len(result)+1)
a = np.array(a)
df = pd.DataFrame({'ind':a,'pre':result})
df.to_csv('./result/ronghe_cnnBilstm616_lr.csv',index = False,header=False) 

# 最终的结果融合

In [4]:
import pandas as pd
import numpy as np
import os
import csv
result1 = './result/ronghe_cnnBilstm616_lr_cate.csv'
result2 = './result/ronghe_cnnBilstm616_lr.csv'
result3 = './result/ronghe_cnnBilstm701_300_100_wiki_lr_cate.csv'
result1_csv = pd.read_csv(result1,delimiter = ',',header = None, index_col=False)
result2_csv = pd.read_csv(result2,delimiter = ',',header = None, index_col=False)
result3_csv = pd.read_csv(result3,delimiter = ',',header = None, index_col=False)
result1_result = result1_csv.loc[:,1].values
result2_result = result2_csv.loc[:,1].values
result3_result = result3_csv.loc[:,1].values
result_2 = ((result1_result==2) + 0)+ ((result2_result==2) + 0) +  ((result3_result==2) + 0)
result_0 =((result1_result==0) + 0)+ ((result2_result==0) + 0) +  ((result3_result==0) + 0)
result = list(map(lambda x,y:2 if x >= 2 else(0 if y >=2 else 1),result_2,result_0))
# 写入结果
with open("./result/result.csv","w+") as outer:
    writer = csv.writer(outer)
    for j in range(len(result)):
        writer.writerow([j+1, result[j]])